In [124]:
import pandas as pd
import numpy as np
from datetime import datetime
from rapidfuzz import process, utils as fuzz_utils
GOOGLE_SHEET_MASTER_LIST_URL = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSWbwrsqF-c---4lfw0LZWymd-f8sy8sLYkXgzh0OyeGATWwrvv7V1Mq5BcApn7F_-WYKP1KXy5shKw/pub?gid=240503925&single=true&output=csv'


In [125]:
master_list_schools_df = pd.read_csv(GOOGLE_SHEET_MASTER_LIST_URL,usecols={'id','california_oid','phone','participating','rowid','school','district_y','address_y'})
# master_list_schools_df.columns# master_list_schools_df = 

master_list_schools_df

,id,california_oid,phone,participating,rowid,school,district_y,address_y
0,0,1.964352e+13,(310) 263-4400,True,306.0,Hawthorne High (Centinela Valley Union High),Centinela Valley Union High,4859 West El Segundo Blvd.
1,1,1.964352e+13,(310) 263-3100,True,307.0,Lawndale High (Centinela Valley Union High),Centinela Valley Union High,14901 South Inglewood Ave.
2,2,1.964352e+11,(310) 263-2200,True,308.0,Leuzinger High (Centinela Valley Union High),Centinela Valley Union High,4118 West Rosecrans Ave.
3,3,1.964352e+11,(310) 263-3264,True,310.0,R. K. Lloyde High (Centinela Valley Union High),Centinela Valley Union High,4951 Marine Ave.
4,4,NaN,(310) 221-0430,True,0.0,Alliance Alice M. Baxter College-Ready High Sc...,NaN,NaN
...,...,...,...,...,...,...,...,...
3703,3703,NaN,NaN,NaN,3138.0,Bert Corona Charter High (Youth Policy Institu...,Youth Policy Institute (YPI) Charter Schools,12513 Gain St.
3704,3704,NaN,NaN,NaN,3139.0,Monsenor Oscar Romero Charter Middle (Youth Po...,Youth Policy Institute (YPI) Charter Schools,2670 West 11th St.
3705,3705,NaN,NaN,NaN,3145.0,YouthBuild - Hollywood (YouthBuild Charter Sch...,YouthBuild Charter Schools of Los Angeles,5941 Hollywood Boulevard
3706,3706,NaN,NaN,NaN,3147.0,YouthBuild - Norwalk (YouthBuild Charter Schoo...,YouthBuild Charter Schools of Los Angeles,12124 Front St.


In [126]:
california_schools_data = "../data/CDESchoolDirectoryExportAugust2022.csv"

california_schools = pd.read_csv(california_schools_data,
    usecols={'CDS Code','County','Status','District','School',"Closed Date","Website","Latitude","Longitude","Last Update",'Street Address','Street City',"Phone","Email"},encoding='latin')
california_schools.columns = ["oid","county","district","name","status","closed_date","website","latitude","longitude","last_update","address","city","default_phone","email"]

california_schools = california_schools.fillna('')
la_schools_df = california_schools[california_schools["county"] == "Los Angeles"]

la_schools_df['original_name'] = la_schools_df['name']
name = la_schools_df['name'].str.lower().str.replace("[[:punct:]]","-",regex=True).replace({'\([a-zA-Z\s\.\-\/]*\(*[a-zA-Z\s\.\-\/]*\)*[a-zA-Z\s\.\-\/]*\)$'},'',regex=True)

la_schools_df.loc[la_schools_df['district'] == "Los Angeles Unified", 'district'] = "LAUSD"
simple_district = la_schools_df['district'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
simple_address = la_schools_df['address'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')

la_schools_df['simple_name'] = simple_address+"-"+name+"-"+simple_district
la_schools_df['simple_name'].fillna('none',inplace=True)

la_schools_df['oid_key'] = la_schools_df['oid'].astype(str).str.replace(".0","")

la_schools_df
# master_list_subset_with_oid['oid_key'] = master_list_subset_with_oid['california_oid'].astype(str).str.replace(".0","")


<ipython-input-126-89e5a086caaa>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  la_schools_df['original_name'] = la_schools_df['name']
c:\Users\hikou\.conda\envs\metro\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-126-89e5a086caaa>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated 

,oid,county,district,name,status,closed_date,website,latitude,longitude,last_update,address,city,default_phone,email,original_name,simple_name,oid_key
0,19642121930346,Los Angeles,ABC Unified,ABC Adult,Active,No Data,http://abcadultschool.edu,33.878924,-118.071286,7/19/2021,12254 Cuesta Dr.,Cerritos,(562) 229-7960,No Data,ABC Adult,12254-cuesta-dr-abc adult-abc-unified,196421219346
1,19642121933225,Los Angeles,ABC Unified,ABC Evening High School,Closed,11/23/1994,No Data,No Data,No Data,6/24/1999,16800 Shoemaker Ave.,Cerritos,No Data,No Data,ABC Evening High School,16800-shoemaker-ave-abc evening high school-ab...,19642121933225
2,19642121995596,Los Angeles,ABC Unified,ABC Secondary (Alternative),Active,No Data,No Data,33.881547,-118.046358,11/5/2021,16534 South Carmenita Rd.,Cerritos,(562) 229-7768,No Data,ABC Secondary (Alternative),16534-south-carmenita-rd-abc secondary -abc-un...,19642121995596
3,19642126138200,Los Angeles,ABC Unified,Accelerated Christian Academy,Closed,5/12/2016,No Data,Information Redacted,Information Redacted,5/12/2016,Information Redacted,Lakewood,Information Redacted,Information Redacted,Accelerated Christian Academy,information-redacted-accelerated christian aca...,196421261380
4,19642126010862,Los Angeles,ABC Unified,Aloha Elementary,Active,No Data,www.alohaes.us,33.835176,-118.083725,11/5/2021,11737 East 214th St.,Lakewood,(562) 229-7825,No Data,Aloha Elementary,11737-east-214th-st-aloha elementary-abc-unified,1964212862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5824,19768696023808,Los Angeles,Wiseburn Unified,Juan De Anza Elementary,Active,No Data,No Data,33.922918,-118.372077,11/5/2021,12110 Hindry Ave.,Hawthorne,(310) 725-2100,No Data,Juan De Anza Elementary,12110-hindry-ave-juan de anza elementary-wiseb...,1976869238
5825,19768690131318,Los Angeles,Wiseburn Unified,Malaga Cove School - Success Learning Center,Closed,6/30/2016,Information Not Available,33.801682,-118.396452,10/24/2016,300 Paseo Del Mar,Palos Verdes Estates,Information Not Available,Information Not Available,Malaga Cove School - Success Learning Center,300-paseo-del-mar-malaga cove school - success...,197686131318
5826,19768696023832,Los Angeles,Wiseburn Unified,Richard Henry Dana Middle,Active,No Data,No Data,33.909404,-118.376654,11/5/2021,5504 West 135th St.,Hawthorne,(310) 725-4700,No Data,Richard Henry Dana Middle,5504-west-135th-st-richard henry dana middle-w...,197686923832
5827,19768690135988,Los Angeles,Wiseburn Unified,RISE High,Closed,9/1/2018,Information Not Available,33.908904,-118.377651,11/30/2018,13500 Aviation Blvd.,Hawthorne,Information Not Available,Information Not Available,RISE High,13500-aviation-blvd-rise high-wiseburn-unified,197686135988


### 2.0 Subset and join data with California IDs


In [127]:
# master_list_subset_no_oid
master_list_subset_with_oid = master_list_schools_df[master_list_schools_df['california_oid'].notnull()]
master_list_subset_with_oid
master_list_subset_with_oid['oid_key'] = master_list_subset_with_oid['california_oid'].astype(str).str.replace(".0","")
master_list_subset_with_oid
inner_join_master_list_with_schools = pd.merge(master_list_subset_with_oid, la_schools_df, left_on='oid_key', right_on='oid_key', how='inner')
inner_join_master_list_with_schools

<ipython-input-127-35bbf3d7aec1>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  master_list_subset_with_oid['oid_key'] = master_list_subset_with_oid['california_oid'].astype(str).str.replace(".0","")
<ipython-input-127-35bbf3d7aec1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master_list_subset_with_oid['oid_key'] = master_list_subset_with_oid['california_oid'].astype(str).str.replace(".0","")


,id,california_oid,phone,participating,rowid,school,district_y,address_y,oid_key,oid,...,website,latitude,longitude,last_update,address,city,default_phone,email,original_name,simple_name
0,0,1.964352e+13,(310) 263-4400,True,306.0,Hawthorne High (Centinela Valley Union High),Centinela Valley Union High,4859 West El Segundo Blvd.,19643521933951,19643521933951,...,No Data,33.916456,-118.362903,11/5/2021,4859 West El Segundo Blvd.,Hawthorne,(310) 263-4400,No Data,Hawthorne High,4859-west-el-segundo-blvd-hawthorne high-centi...
1,1,1.964352e+13,(310) 263-3100,True,307.0,Lawndale High (Centinela Valley Union High),Centinela Valley Union High,14901 South Inglewood Ave.,19643521934926,19643521934926,...,No Data,33.896373,-118.361369,6/30/2022,14901 South Inglewood Ave.,Lawndale,(310) 263-3102,No Data,Lawndale High,14901-south-inglewood-ave-lawndale high-centin...
2,2,1.964352e+11,(310) 263-2200,True,308.0,Leuzinger High (Centinela Valley Union High),Centinela Valley Union High,4118 West Rosecrans Ave.,196435219348,19643521935048,...,No Data,33.901370,-118.346870,11/5/2021,4118 West Rosecrans Ave.,Lawndale,(310) 263-2208,No Data,Leuzinger High,4118-west-rosecrans-ave-leuzinger high-centine...
3,3,1.964352e+11,(310) 263-3264,True,310.0,R. K. Lloyde High (Centinela Valley Union High),Centinela Valley Union High,4951 Marine Ave.,196435219239,19643521930239,...,No Data,33.895165,-118.365993,11/5/2021,4951 Marine Ave.,Lawndale,(310) 263-3264,No Data,R. K. Lloyde High,4951-marine-ave-r. k. lloyde high-centinela-va...
4,5,1.964731e+11,(323) 744-2122,True,91.0,Alliance Cindy and Bill Simon Technology Acade...,Alliance College Ready Public Schools District,10720 South Wilmington Ave.,196473121285,19647330121285,...,http://simontechnology.org,33.938494,-118.238742,4/11/2022,10720 Wilmington Ave.,Los Angeles,(323) 744-2122,No Data,Alliance Cindy and Bill Simon Technology High ...,10720-wilmington-ave-alliance cindy and bill s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,1479,1.964881e+11,(626) 396-5910,True,2685.0,Sierra Madre Middle (Pasadena Unified),Pasadena Unified,160 North Canon Ave.,196488127746,19648810127746,...,http://smms.pusd.us,34.164700,-118.044910,11/5/2021,160 North Canon Ave.,Sierra Madre,(626) 396-5910,No Data,Sierra Madre Middle,160-north-canon-ave-sierra madre middle-pasade...
1425,1480,1.964882e+13,(626) 396-5840,True,221.0,Washington Elementary (Bellflower Unified),Bellflower Unified,9725 East Jefferson St.,19648816119549,19648816119549,...,http://washington.pusd.us,34.171222,-118.149053,11/5/2021,1520 North Raymond Ave.,Pasadena,(626) 396-5840,No Data,Washington Elementary,1520-north-raymond-ave-washington elementary-p...
1426,1481,1.964881e+11,(626) 396-5690,True,1143.0,Willard Elementary (Long Beach Unified),Long Beach Unified,1055 Freeman Ave.,196488121778,19648816021778,...,http://willard.pusd.us/,34.142174,-118.081616,11/5/2021,301 South Madre St.,Pasadena,(626) 396-5690,No Data,Willard Elementary,301-south-madre-st-willard elementary-pasadena...
1427,1482,1.964734e+13,(323) 441-3113,True,862.0,Cathedral High School (Independent Charter Sch...,Independent Charter Schools District,1253 Bishops Rd.,19647336934442,19647336934442,...,No Data,34.069854,-118.234286,6/27/2022,1253 Bishops Rd.,Los Angeles,(323) 225-2438,brjohnm@chsla.org,Cathedral High School,1253-bishops-rd-cathedral high school-lausd


In [128]:
# inner_join_master_list_with_schools.loc[inner_join_master_list_with_schools['phone'].isnull(), 'participating'] 
inner_join_master_list_with_schools.loc[inner_join_master_list_with_schools['phone'].isnull()]
# null_phone_df  = inner_join_master_list_with_schools[inner_join_master_list_with_schools['phone'].str.len() < 15]

inner_join_master_list_with_schools.loc[inner_join_master_list_with_schools['phone'].isnull(), 'phone'] = inner_join_master_list_with_schools['default_phone']
# null_phone_df  = inner_join_master_list_with_schools[inner_join_master_list_with_schools['phone'].str.contains("No Data")]
# null_phone_df
# inner_join_master_list_with_schools.columns
inner_join_master_list_with_schools_df_to_combine = inner_join_master_list_with_schools[['california_oid','phone','participating','rowid','school','district_y','address_y','status','latitude','longitude','email','website']]
inner_join_master_list_with_schools_df_to_combine

,california_oid,phone,participating,rowid,school,district_y,address_y,status,latitude,longitude,email,website
0,1.964352e+13,(310) 263-4400,True,306.0,Hawthorne High (Centinela Valley Union High),Centinela Valley Union High,4859 West El Segundo Blvd.,Active,33.916456,-118.362903,No Data,No Data
1,1.964352e+13,(310) 263-3100,True,307.0,Lawndale High (Centinela Valley Union High),Centinela Valley Union High,14901 South Inglewood Ave.,Active,33.896373,-118.361369,No Data,No Data
2,1.964352e+11,(310) 263-2200,True,308.0,Leuzinger High (Centinela Valley Union High),Centinela Valley Union High,4118 West Rosecrans Ave.,Active,33.901370,-118.346870,No Data,No Data
3,1.964352e+11,(310) 263-3264,True,310.0,R. K. Lloyde High (Centinela Valley Union High),Centinela Valley Union High,4951 Marine Ave.,Active,33.895165,-118.365993,No Data,No Data
4,1.964731e+11,(323) 744-2122,True,91.0,Alliance Cindy and Bill Simon Technology Acade...,Alliance College Ready Public Schools District,10720 South Wilmington Ave.,Active,33.938494,-118.238742,No Data,http://simontechnology.org
...,...,...,...,...,...,...,...,...,...,...,...,...
1424,1.964881e+11,(626) 396-5910,True,2685.0,Sierra Madre Middle (Pasadena Unified),Pasadena Unified,160 North Canon Ave.,Active,34.164700,-118.044910,No Data,http://smms.pusd.us
1425,1.964882e+13,(626) 396-5840,True,221.0,Washington Elementary (Bellflower Unified),Bellflower Unified,9725 East Jefferson St.,Active,34.171222,-118.149053,No Data,http://washington.pusd.us
1426,1.964881e+11,(626) 396-5690,True,1143.0,Willard Elementary (Long Beach Unified),Long Beach Unified,1055 Freeman Ave.,Active,34.142174,-118.081616,No Data,http://willard.pusd.us/
1427,1.964734e+13,(323) 441-3113,True,862.0,Cathedral High School (Independent Charter Sch...,Independent Charter Schools District,1253 Bishops Rd.,Active,34.069854,-118.234286,brjohnm@chsla.org,No Data


### 3.0 Subset the data with no California IDs


In [129]:
master_list_subset_no_oid = master_list_schools_df[master_list_schools_df['california_oid'].isnull()]
name = master_list_subset_no_oid['school'].str.replace("[[:punct:]]","-",regex=True).replace({'\([a-zA-Z\s\.\-\/]*\(*[a-zA-Z\s\.\-\/]*\)*[a-zA-Z\s\.\-\/]*\)$'},'',regex=True)

simple_district = master_list_subset_no_oid['district_y'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
simple_address = master_list_subset_no_oid['address_y'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
simple_name = name.str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','').str.split().str.get(0)
key = simple_address+"-"+simple_name
clean_key = key.str.replace("---","-")
master_list_subset_no_oid['tap_key'] = clean_key
master_list_subset_no_oid['tap_key'].fillna('none',inplace=True)
master_list_subset_no_oid

<ipython-input-129-59912ee4e85d>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  simple_district = master_list_subset_no_oid['district_y'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
<ipython-input-129-59912ee4e85d>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  simple_address = master_list_subset_no_oid['address_y'].str.lower().str.replace(" ","-").str.replace(".","").str.replace('"','')
<ipython-input-129-59912ee4e85d>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  simple_name = name.s

,id,california_oid,phone,participating,rowid,school,district_y,address_y,tap_key
4,4,NaN,(310) 221-0430,True,0.0,Alliance Alice M. Baxter College-Ready High Sc...,NaN,NaN,none
6,6,NaN,(323) 920-4388,True,90.0,Alliance Virgil Roberts Leadership Academy (Al...,Alliance College Ready Public Schools District,2941 West 70th St.,2941-west-70th-st-alliance-virgil-roberts-lead...
7,7,NaN,(323) 565-1600,True,680.0,Animo City of Champions Charter High (Green Do...,Green Dot Charter Schools District,9330 South Eighth Ave.,9330-south-eighth-ave-animo-city-of-champions-...
8,8,NaN,(323) 565-3280,True,682.0,Animo Florence-Firestone Charter Middle (Green...,Green Dot Charter Schools District,8255 Beach St.,8255-beach-st-animo-florence-firestone-charter...
9,9,NaN,(323) 565-2100,True,676.0,Animo Inglewood Charter High (Green Dot Charte...,Green Dot Charter Schools District,3425 West Manchester Blvd.,3425-west-manchester-blvd-animo-inglewood-char...
...,...,...,...,...,...,...,...,...,...
3703,3703,NaN,NaN,NaN,3138.0,Bert Corona Charter High (Youth Policy Institu...,Youth Policy Institute (YPI) Charter Schools,12513 Gain St.,12513-gain-st-bert-corona-charter-high-
3704,3704,NaN,NaN,NaN,3139.0,Monsenor Oscar Romero Charter Middle (Youth Po...,Youth Policy Institute (YPI) Charter Schools,2670 West 11th St.,2670-west-11th-st-monsenor-oscar-romero-charte...
3705,3705,NaN,NaN,NaN,3145.0,YouthBuild - Hollywood (YouthBuild Charter Sch...,YouthBuild Charter Schools of Los Angeles,5941 Hollywood Boulevard,5941-hollywood-boulevard-youthbuild-hollywood-
3706,3706,NaN,NaN,NaN,3147.0,YouthBuild - Norwalk (YouthBuild Charter Schoo...,YouthBuild Charter Schools of Los Angeles,12124 Front St.,12124-front-st-youthbuild-norwalk-


In [130]:

def fuzzy_merge(baseFrame, compareFrame, baseKey, compareKey, threshold=86, limit=1, how='left'):
    s_mapping = {x: fuzz_utils.default_process(x) for x in compareFrame[compareKey]}

    m1 = baseFrame[baseKey].apply(lambda x: process.extract(
      fuzz_utils.default_process(x), s_mapping, limit=limit, score_cutoff=threshold, processor=None
    ))
    baseFrame['Match'] = m1

    m2 = baseFrame['Match'].apply(lambda x: ', '.join(i[2] for i in x))
    baseFrame[baseKey] = m2.replace("",np.nan)

    return baseFrame.merge(compareFrame, left_on=baseKey, right_on=compareKey, how=how)

merged_df = fuzzy_merge(master_list_subset_no_oid, la_schools_df, 'tap_key', 'simple_name',how='left')
# merged_df = fuzzy_merge(la_schools_df, go_pass_schools 'name', 'simple_name',how='right')
merged_df

In [ ]:
# merged_df.columns
# merged_df_to_combine = merged_df[['id','california_oid','phone','participating','rowid','school','district_y','address_y','latitude','longitude','email']]
merged_df.loc[merged_df['phone'].isnull(), 'phone'] = merged_df['default_phone']
merged_df_to_combine = merged_df[['phone','participating','rowid','school','district_y','address_y','Match','oid','status','latitude','longitude','email','website']]

merged_df_to_combine.rename(columns = {'oid':'california_oid'}, inplace = True)
merged_df_to_combine


# merged_df_to_combine

c:\Users\hikou\.conda\envs\metro\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,phone,participating,rowid,school,district_y,address_y,Match,california_oid,status,latitude,longitude,email,website
0,(310) 221-0430,True,0.0,Alliance Alice M. Baxter College-Ready High Sc...,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN
1,(323) 920-4388,True,90.0,Alliance Virgil Roberts Leadership Academy (Al...,Alliance College Ready Public Schools District,2941 West 70th St.,[(2941 west 70th st alliance virgil roberts le...,19647330128009,Active,33.977090,-118.325758,No Data,www.avrlacademy.org
2,(323) 565-1600,True,680.0,Animo City of Champions Charter High (Green Do...,Green Dot Charter Schools District,9330 South Eighth Ave.,[(9330 south eighth ave animo city of champion...,19101990136119,Active,33.951586,-118.325055,cha.cde@greendot.org,http://greendot.org
3,(323) 565-3280,True,682.0,Animo Florence-Firestone Charter Middle (Green...,Green Dot Charter Schools District,8255 Beach St.,[(8255 beach st animo florence firestone chart...,19647330134023,Active,33.962984,-118.242612,flo.cde@greendot.org,http://ca.greendot.org/florence-firestone/
4,(323) 565-2100,True,676.0,Animo Inglewood Charter High (Green Dot Charte...,Green Dot Charter Schools District,3425 West Manchester Blvd.,[(3425 west manchester blvd animo inglewood ch...,19646341996586,Active,33.960029,-118.331386,ing.cde@greendot.org,www.greendot.org/inglewood
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2567,NaN,NaN,3138.0,Bert Corona Charter High (Youth Policy Institu...,Youth Policy Institute (YPI) Charter Schools,12513 Gain St.,[],NaN,NaN,NaN,NaN,NaN,NaN
2568,(213) 413-9600,NaN,3139.0,Monsenor Oscar Romero Charter Middle (Youth Po...,Youth Policy Institute (YPI) Charter Schools,2670 West 11th St.,[(2670 west 11th st monsenor oscar romero char...,19647330114959,Active,34.050819,-118.294337,No Data,http://morcs.ypics.org/
2569,NaN,NaN,3145.0,YouthBuild - Hollywood (YouthBuild Charter Sch...,YouthBuild Charter Schools of Los Angeles,5941 Hollywood Boulevard,[],NaN,NaN,NaN,NaN,NaN,NaN
2570,NaN,NaN,3147.0,YouthBuild - Norwalk (YouthBuild Charter Schoo...,YouthBuild Charter Schools of Los Angeles,12124 Front St.,[],NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
combined_df = pd.concat([inner_join_master_list_with_schools_df_to_combine, merged_df_to_combine], ignore_index=True)
combined_df.reset_index(drop=True, inplace=True)
combined_df.index.names = ['id']
combined_df['california_oid'] = combined_df['california_oid'].astype(str).str.replace(".0","")
combined_df['rowid'] = combined_df['rowid'].astype(str).str.replace(".0","")
combined_df



<ipython-input-123-f319580a22c5>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  combined_df['california_oid'] = combined_df['california_oid'].astype(str).str.replace(".0","")
<ipython-input-123-f319580a22c5>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  combined_df['rowid'] = combined_df['rowid'].astype(str).str.replace(".0","")


,california_oid,phone,participating,rowid,school,district_y,address_y,status,latitude,longitude,email,website,Match
id,,,,,,,,,,,,,
0,19643521933951,(310) 263-4400,True,6,Hawthorne High (Centinela Valley Union High),Centinela Valley Union High,4859 West El Segundo Blvd.,Active,33.916456,-118.362903,No Data,No Data,NaN
1,19643521934926,(310) 263-3100,True,7,Lawndale High (Centinela Valley Union High),Centinela Valley Union High,14901 South Inglewood Ave.,Active,33.896373,-118.361369,No Data,No Data,NaN
2,196435219348,(310) 263-2200,True,8,Leuzinger High (Centinela Valley Union High),Centinela Valley Union High,4118 West Rosecrans Ave.,Active,33.901370,-118.346870,No Data,No Data,NaN
3,196435219239,(310) 263-3264,True,3,R. K. Lloyde High (Centinela Valley Union High),Centinela Valley Union High,4951 Marine Ave.,Active,33.895165,-118.365993,No Data,No Data,NaN
4,196473121285,(323) 744-2122,True,91,Alliance Cindy and Bill Simon Technology Acade...,Alliance College Ready Public Schools District,10720 South Wilmington Ave.,Active,33.938494,-118.238742,No Data,http://simontechnology.org,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3996,nan,NaN,NaN,3138,Bert Corona Charter High (Youth Policy Institu...,Youth Policy Institute (YPI) Charter Schools,12513 Gain St.,NaN,NaN,NaN,NaN,NaN,[]
3997,196473114959,(213) 413-9600,NaN,3139,Monsenor Oscar Romero Charter Middle (Youth Po...,Youth Policy Institute (YPI) Charter Schools,2670 West 11th St.,Active,34.050819,-118.294337,No Data,http://morcs.ypics.org/,[(2670 west 11th st monsenor oscar romero char...
3998,nan,NaN,NaN,3145,YouthBuild - Hollywood (YouthBuild Charter Sch...,YouthBuild Charter Schools of Los Angeles,5941 Hollywood Boulevard,NaN,NaN,NaN,NaN,NaN,[]


In [ ]:
output_file_name = "../data/final_tap_data_post_processing_master_list.csv"
combined_df.to_csv(output_file_name)

output_json = "../src/data/schools.json"
json_file = combined_df.to_json(orient='records',index=True) 
with open(output_json, 'w') as f:
    f.write(json_file)